In [47]:
import pandas as pd 
import string 
from nltk.corpus import stopwords


In [48]:
df_spamcollection = pd.read_csv('C:/Users/Suri/Desktop/data_science/simply_learn/lesson91/Lesson_9_1/SpamCollection',sep ='\t',names = ['response','message'])

In [49]:
df_spamcollection.head()

,response,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [50]:
#view more information about the spam data using describe method
df_spamcollection.describe()

,response,message
count,5572,5572
unique,2,5169
top,ham,"Sorry, I'll call later"
freq,4825,30


In [51]:
#group by and describe method
df_spamcollection['length']= df_spamcollection['message'].apply(len)
df_spamcollection.head()

,response,message,length
0,ham,"Go until jurong point, crazy.. Available only ...",111
1,ham,Ok lar... Joking wif u oni...,29
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155
3,ham,U dun say so early hor... U c already then say...,49
4,ham,"Nah I don't think he goes to usf, he lives aro...",61


In [52]:
#def a function to get rid of stopwords present in the massage 
def message_text(mess):
    #chack character to see if there are punctuation 
    no_punctuation = [char for char in mess if char not in string.punctuation]
    #now form the sentance 
    no_punctuation = ''.join(no_punctuation)
    #now eleminate any stopwords 
    return [ word for word in no_punctuation.split() if word.lower() not in stopwords.words('english')]

In [57]:
#verify the function is working 
df_spamcollection['message'].head().apply(message_text)

0    [Go, jurong, point, crazy, Available, bugis, n...
1                       [Ok, lar, Joking, wif, u, oni]
2    [Free, entry, 2, wkly, comp, win, FA, Cup, fin...
3        [U, dun, say, early, hor, U, c, already, say]
4    [Nah, dont, think, goes, usf, lives, around, t...
Name: message, dtype: object

In [58]:
#text processing with vectorizer
from sklearn.feature_extraction.text import CountVectorizer 

In [61]:
bag_of_words = CountVectorizer( analyzer = message_text).fit(df_spamcollection['message'])

In [62]:
print(len(bag_of_words.vocabulary_))

11425


In [64]:
msg_bagofwords = bag_of_words.transform (df_spamcollection['message'])

In [66]:
#apply tfidf transfore and fit the bag of words into it ( transformed version)
from sklearn.feature_extraction.text import TfidfTransformer 

In [67]:
tfidf_transformer =  TfidfTransformer ().fit(msg_bagofwords)

In [68]:
#print shape of the tfidf
msg_tfidf = tfidf_transformer.transform(msg_bagofwords)

In [70]:
print(msg_tfidf.shape)

(5572, 11425)


In [74]:
#choose naive based model to detect the spam and fit tfidf data into it 
from sklearn.naive_bayes import MultinomialNB
spam_detect_model = MultinomialNB().fit(msg_tfidf,df_spamcollection['response'])

In [86]:
#chack model for the predicted and expected value say for message#2 and message#5
message = df_spamcollection['message'][2]
bag_of_words_for_msg = bag_of_words.transform([message])
tfidf = tfidf_transformer.transform(bag_of_words_for_msg)
                                      
print('predicted',spam_detect_model.predict(tfidf)[0])
print('expected',df_spamcollection.response[2])

predicted spam
expected spam
